In [3]:
import stockutils
import pandas_datareader.data as web, re, sqlite3, datetime

In [50]:
class StockData:
    def __init__(self, se, update = False):
        self.company_table = stockutils.company_table(se = se, update = update)
        self.se = se
   
    def lookup_ticker(self, company_name, sql = True):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        if isinstance(company_name,str):
            company_name = [company_name]
        result_table = [] 
        if not sql:
            for name in company_name:
                name = r'.*\s*'+name.lower()+r'.*'
                for row in self.company_table:
                    match = re.search(name,row[1].lower()) 
                    if match:
                        result_table.append((row[0],row[1]))
        else:
            conn = sqlite3.connect('data/companysql' + self.se + '.sql')
            c = conn.cursor()
            for name in company_name:
                query = ('%'+name+'%',)
                for row in c.execute('SELECT symbol,name FROM companytable WHERE lower(name) LIKE ?', query):
                    result_table.append(row)
            conn.close()
            if not result_table:
                print('Switching to table.')
                return self.lookup_ticker(company_name, sql = False)
        return result_table.pop() if len(result_table) == 1 else result_table
    
#    def stock_price(self, ticker, date):
        
        

In [32]:
x = StockData('nasdaq')

In [51]:
y = StockData('amex')

In [52]:
z = StockData('nyse')

In [46]:
x.lookup_ticker('Nvidia')

('NVDA', 'NVIDIA Corporation')

In [47]:
x.lookup_ticker(['Intel Corporation', 'Nvidia', 'Apple', 'Microsoft', 'Becton Dickinson'])

[('INTC', 'Intel Corporation'),
 ('NVDA', 'NVIDIA Corporation'),
 ('AAPL', 'Apple Inc.'),
 ('MSFT', 'Microsoft Corporation')]

In [53]:
z.lookup_ticker("Sporting")

('DKS', 'Dick&#39;s Sporting Goods Inc')

In [54]:
z.lookup_ticker('Becton')

('BDX', 'Becton, Dickinson and Company')